In [2]:
import pandas as pd

In [3]:
sentiment = pd.read_csv('movie-reviews/train.tsv',sep='\t',index_col=['PhraseId'])

In [4]:
cleaned_sentiment = sentiment[sentiment.Sentiment != 3]

In [5]:
cleaned_sentiment.tail()

,SentenceId,Phrase,Sentiment
PhraseId,,,
156055,8544,Hearst 's forced avuncular chortles,2
156056,8544,Hearst 's,2
156057,8544,forced avuncular chortles,1
156059,8544,avuncular,2
156060,8544,chortles,2


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [46]:
vec = TfidfVectorizer(stop_words='english',ngram_range=(1,3))

In [47]:
vec_comment = vec.fit_transform(cleaned_sentiment['Phrase'])

In [48]:
from sklearn.naive_bayes import MultinomialNB

In [49]:
model = MultinomialNB()

In [50]:
model.fit( vec_comment, cleaned_sentiment['Sentiment'])

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [51]:
test_data = pd.read_csv('movie-reviews/test.tsv',sep='\t')

In [52]:
test_data.set_index(['PhraseId'],inplace=True)

In [53]:
test_data.head()

,SentenceId,Phrase
PhraseId,,
156061,8545,An intermittently pleasing but mostly routine ...
156062,8545,An intermittently pleasing but mostly routine ...
156063,8545,An
156064,8545,intermittently pleasing but mostly routine effort
156065,8545,intermittently pleasing but mostly routine


In [54]:
o = vec.transform([test_data.iloc[0]['Phrase']])

In [55]:
vec_out = vec.transform(test_data['Phrase'])

In [56]:
pred = model.predict(vec_out)

In [57]:
pred.shape

(66292,)

In [58]:
import numpy as np

In [59]:
np.where(pred == 1)

(array([  481,   628,   629, ..., 66245, 66246, 66247], dtype=int64),)

In [60]:
test_data.iloc[301]

SentenceId                                                 8553
Phrase        The film contains no good jokes , no good scen...
Name: 156362, dtype: object

In [61]:
test_data['pred'] = pred

In [66]:
test_data.reset_index(inplace=True)

out = test_data[['PhraseId','Sentiment']]

KeyError: "['Sentiment'] not in index"

In [63]:
out.to_csv('sub.csv')

In [64]:
x = out.set_index(['PhraseId'])

In [65]:
x.to_csv('a.csv')

In [67]:
#Splitting data into train-test
from sklearn.model_selection import train_test_split

In [75]:
X_train, X_test, Y_train, Y_test = train_test_split(cleaned_sentiment[['Phrase']],cleaned_sentiment['Sentiment'])

In [76]:
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier as knn
from sklearn.naive_bayes import GaussianNB as GB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score 

Classification_models = [RandomForestClassifier(n_estimators=1000),MultinomialNB(),knn(n_neighbors=7),LogisticRegression(),SVC()]

In [ ]:
vec_comment = vec.fit_transform(X_train['Phrase'])
vec_p = vec.transform(X_test['Phrase'])

for model in Classification_models:
    pred = model.fit(vec_comment,Y_train).predict(vec_p)
    print (accuracy_score(Y_test,pred))